In [6]:
import numpy as np
from sklearn import datasets 
datas = datasets.load_digits()

In [7]:
datas.keys()

dict_keys(['data', 'target', 'target_names', 'images', 'DESCR'])

In [8]:
X = datas.data
y = datas.target

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

/home/crystal/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# 1.用循环来搜寻合适的参数

In [11]:

from sklearn.neighbors import KNeighborsClassifier
best_score, best_k, best_p = 0, 0, 0
for k in range(2,11):
    for p in range(1,6):
        KnnClf = KNeighborsClassifier(n_neighbors=k, weights='distance', p=p)
        KnnClf.fit(X_train, y_train)
        score = KnnClf.score(X_test, y_test)
        if score > best_score: 
            best_score, best_k, best_p = score, k, p

print(best_score, best_k, best_p)
            

/home/crystal/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


0.9860917941585535 3 4


# 2.(1)用sklearn中的Cross-validation 的方法来搜寻最佳的超参数

In [15]:

from sklearn.model_selection import cross_val_score
best_score, best_k, best_p = 0, 0, 0
for k in range(2,11):
    for p in range(1,6):
        KnnClf1 = KNeighborsClassifier(n_neighbors=k, weights='distance', p=p)
        score = cross_val_score(KnnClf1, X_train, y_train)
        mean_score = np.mean(score)
        if mean_score > best_score: 
            best_score, best_k, best_p = mean_score, k, p

print(best_score, best_k, best_p)

0.9823599874006478 2 2


In [53]:
KnnClf1_best = KNeighborsClassifier(n_neighbors=2, weights='distance', p=2)
KnnClf1_best.fit(X_train,y_train)
KnnClf1_best.score(X_test,y_test)

0.980528511821975

# (2)自己实现一个交叉验证函数

In [61]:

def mycross_val_score(estimator, X_train, y_train):
    n = len(X_train)
    index = np.arange(n)
    np.random.shuffle(index)
    X_train1 = X_train[index[:n//3],:]
    y_train1 = y_train[index[:n//3]]
    X_train2 = X_train[index[n//3:2*n//3],:]
    y_train2 = y_train[index[n//3:2*n//3]]
    X_train3 = X_train[index[2*n//3:],:]
    y_train3 = y_train[index[2*n//3:]]
    
    score = []
    estimator.fit(np.vstack([X_train1, X_train2]), np.hstack([y_train1, y_train2]))
    score.append(estimator.score(X_train3, y_train3))
    estimator.fit(np.vstack([X_train1, X_train3]), np.hstack([y_train1, y_train3]))
    score.append(estimator.score(X_train2, y_train2))
    estimator.fit(np.vstack([X_train2, X_train3]), np.hstack([y_train2, y_train3]))
    score.append(estimator.score(X_train1, y_train1))
    return score

best_score, best_k, best_p = 0, 0, 0
for k in range(2,11):
    for p in range(1,6):
        KnnClf1 = KNeighborsClassifier(n_neighbors=k, weights='distance', p=p)
        score = mycross_val_score(KnnClf1, X_train, y_train)
        mean_score = np.mean(score)
        if mean_score > best_score: 
            best_score, best_k, best_p = mean_score, k, p

print(best_score, best_k, best_p)

0.9842308882698854 4 2


In [62]:
KnnClf1_best = KNeighborsClassifier(n_neighbors=4, weights='distance', p=2)
KnnClf1_best.fit(X_train,y_train)
KnnClf1_best.score(X_test,y_test)

0.9847009735744089

# 3.用sklearn中GridSearchCV网格搜索的方法来搜寻最佳的超参数

In [71]:
# GridSearchCV 中的CV指的便是Cross-validation（交叉验证）
from sklearn.model_selection import GridSearchCV

parameters = {'weights' : ['distance'],
              'n_neighbors':[i for i in range(2,11)], 
              'p':[i for i in range(1,6)]}
model = KNeighborsClassifier()
KnnClf2 = GridSearchCV(model, parameters, verbose=1)
KnnClf2.fit(X_train, y_train)

KnnClf2.best_estimator_ 


Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  6.1min finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance')

Fitting 3 folds for each of 45 candidates, totalling 135 fits的含义是：   
将X_train 分成了3层, 参数的组合总共有9×6=45, 总共需要拟合3×45=135次    
注意：默认的是将X_train 分成三份,根据不同的需要,可以通过调整cv这个参数,来决定将X_train分成几分   
eg：KnnClf2 = GridSearchCV(model, parameters, verbose=1, cv=5) # 即将传入的数据分成了5份    
同理：cross_val_score(KnnClf1, X_train, y_train, cv=5)  # cv =5, 即将传入的数据分成了5份   


In [68]:
KnnClf2.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [69]:
KnnClf2.best_score_

0.9823747680890538

In [70]:
KnnClf2_best = KnnClf2.best_estimator_ 
KnnClf2_best.fit(X_train, y_train)
KnnClf2_best.score(X_test,y_test)

0.980528511821975

# 可以看到,用sklearn中GridSearchCV与2.(1)用sklearn中的Cross-validation 结果相同,再次说明了GridSearchCV中的CV即为Cross-validation